In [3]:
# GENDER
import torch
import clip
from PIL import Image
from torch import nn

# используем cuda для распараллеливания вычислений на gpu
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device)


class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(self, x):
        return self.network(x)


model_head = MyModel()
model_head.load_state_dict(torch.load("gender/model_head_weights_celeba_v3_1e-5.pth"))
model_head.eval()


# Функция для распознавания биологической гендерной идентификации
def predict_gender(image):
    image_input = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_input)

    image_features /= image_features.norm(dim=-1, keepdim=True)
    image_features = image_features.to(dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model_head(image_features)

    prob = torch.sigmoid(output)

    return 1 if prob >= 0.5 else 0

In [4]:
# FACE
import onnxruntime
import numpy as np
import cv2
import mediapipe as mp

mp_face_detection = (
    mp.solutions.face_detection
)  # Импорт модуля mediapipe для распознавания лиц

mp_drawing = mp.solutions.drawing_utils  # Утилиты от mediapipe для отрисовки на изображениях
ort_session = onnxruntime.InferenceSession("face/cpu.onnx")  # Загрузка ONNX модели
# Определение списка атрибутов лица для модели
list_attr_en = np.array(
    [
        "5_o_Clock_Shadow",
        "Arched_Eyebrows",
        "Attractive",
        "Bags_Under_Eyes",
        "Bald",
        "Bangs",
        "Big_Lips",
        "Big_Nose",
        "Black_Hair",
        "Blond_Hair",
        "Blurry",
        "Brown_Hair",
        "Bushy_Eyebrows",
        "Chubby",
        "Double_Chin",
        "Eyeglasses",
        "Goatee",
        "Gray_Hair",
        "Heavy_Makeup",
        "High_Cheekbones",
        "Hair",
        "Mouth_Slightly_Open",
        "Mustache",
        "Narrow_Eyes",
        "No_Beard",
        "Oval_Face",
        "Pale_Skin",
        "Pointy_Nose",
        "Receding_Hairline",
        "Rosy_Cheeks",
        "Sideburns",
        "Smiling",
        "Straight_Hair",
        "Wavy_Hair",
        "Wearing_Earrings",
        "Wearing_Hat",
        "Wearing_Lipstick",
        "Wearing_Necklace",
        "Wearing_Necktie",
        "Young",
    ]
)


# Процедура предварительной обработки изображения под нужную модели размерность и нормализацию
def cv2_preprocess(img):
    img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_NEAREST)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mean = [0.5, 0.5, 0.5]
    std = [0.5, 0.5, 0.5]
    img = (img / 255.0 - mean) / std
    img = img.transpose((2, 0, 1))
    img = np.expand_dims(img, axis=0)
    img = np.ascontiguousarray(img, dtype=np.float32)
    return img

# Функция для вычисления сигмоиды
def sigmoid_array(x):
    return 1 / (1 + np.exp(-x))

# Функция проходит через ONNX-модель и возвращает возможности (вероятности) для различных атрибутов лица
def result_inference(input_array):
    ort_inputs = {ort_session.get_inputs()[0].name: input_array}
    ort_outs = ort_session.run(None, ort_inputs)
    possibility = sigmoid_array(ort_outs[0]) > 0.5
    result = list_attr_en[possibility[0]]
    return result

2024-03-30 22:40:27.183573: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# AGE
import cv2
import numpy as np
import tensorflow as tf

age_model = tf.keras.models.load_model("age/model")


def predict_age(image):
    image = cv2.resize(np.array(image), (224, 224)) / 255.0
    image = np.expand_dims(image, axis=0)
    return (age_model.predict(image)[0][0]).astype("int")

In [8]:
# АНАЛИЗ ИЗОБРАЖЕНИЯ

# Функции выполняют обработку изображения, распознавание лиц на нём, а также определение пола и возраста каждого обнаруженного лица.
# 1. square: Эта функция получает обнаруженное лицо и исходное изображение, затем она вычисляет координаты верхнего левого и нижнего правого углов бокса, ограничивающего это лицо. Эти координаты используются для выделения области изображения, содержащей лицо.
# 2. draw: Эта функция принимает обнаруженные лица и исходное изображение. Для каждого обнаруженного лица она рисует бокс, вычисленный функцией square, а также получает и обрабатывает область изображения внутри этого бокса. Обработка включает в себя препроцессинг, получение информации о лице (атрибуты, пол и возраст) и запись этой информации непосредственно на исходное изображение.
# 3. process_image: Эта функция получает путь к изображению. Она считывает изображение, обрабатывает его с помощью модели обнаружения лиц, затем передаёт обнаруженные лица и исходное изображение функции draw для дальнейшей обработки и аннотации.

# Подсчет координат для отрисовки прямоугольника, обозначающего лицо на изображении
def square(detection, image):
    image_rows, image_cols, _ = image.shape
    location = detection.location_data.relative_bounding_box
    start_point = mp_drawing._normalized_to_pixel_coordinates(
        location.xmin, location.ymin, image_cols, image_rows
    )
    end_point = mp_drawing._normalized_to_pixel_coordinates(
        location.xmin + location.width,
        location.ymin + location.height,
        image_cols,
        image_rows,
    )
    x1, y1 = start_point
    x2, y2 = end_point
    return x1, y1, x2, y2

# Функция для отрисовки рамок вокруг обнаруженных лиц и вывода результатов модели на изображении
def draw(results, image):
    if results is not None:
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image2 = image.copy()
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
                x1, y1, x2, y2 = square(detection, image)
                img_infer = image2[y1 - 70 : y2, x1 - 50 : x2 + 50].copy()
                img_infer2 = cv2_preprocess(img_infer)
                result = result_inference(img_infer2)

                # Преобразование numpy изображения в PIL
                img_infer_pil = Image.fromarray(img_infer)

                gender = predict_gender(img_infer_pil)
                gender_text = "female" if gender == 0 else "male"
                result = np.append(result, f"gender: {gender_text}")

                age = predict_age(img_infer_pil)
                result = np.append(result, f"age: {age}")

                for i, text in enumerate(result):
                    image = cv2.putText(
                        image,
                        text,
                        (x1, y1 + i * 40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (255, 255, 255),
                        1,
                        cv2.LINE_AA,
                    )
            cv2.imwrite("output.jpg", image)

# Функция для обработки изображения: считывания изображения, применения модели распознавания лиц и отрисовки результатов
def process_image(image_path):
    image = cv2.imread(image_path)
    with mp_face_detection.FaceDetection(
        model_selection=1, min_detection_confidence=0.5
    ) as face_detection:
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)
        draw(results, image)


# process_image("face/test_img.jpg")
# process_image("man1.jpg")
# process_image("kid1.jpg")
process_image("girl1.jpg")
# process_image("girl2.jpg")

1/1 [==============================] - 0s 111ms/step


In [7]:

cap = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection(
    model_selection=1, min_detection_confidence=0.5
) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            break
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image2 = image.copy()
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
                image_rows, image_cols, _ = image.shape
                location = detection.location_data.relative_bounding_box
                start_point = mp_drawing._normalized_to_pixel_coordinates(
                    location.xmin, location.ymin, image_cols, image_rows
                )
                end_point = mp_drawing._normalized_to_pixel_coordinates(
                    location.xmin + location.width,
                    location.ymin + location.height,
                    image_cols,
                    image_rows,
                )

                if end_point is not None:
                    x2, y2 = end_point
                    x1, y1 = start_point[0], start_point[1]

                    # Добавляем проверку на размер изображения
                    y1_new = max(0, y1 - 70)
                    y2_new = min(image.shape[0], y2)
                    x1_new = max(0, x1 - 50)
                    x2_new = min(image.shape[1], x2 + 50)

                    img_infer = image2[y1_new:y2_new, x1_new:x2_new].copy()

                    # Проверяем, что изображение не пустое
                    if img_infer.size != 0:
                        img_infer2 = cv2_preprocess(img_infer)
                        result = result_inference(img_infer2)

                        # Преобразование numpy изображения в PIL
                        img_infer_pil = Image.fromarray(img_infer)

                        gender = predict_gender(img_infer_pil)
                        gender_text = "female" if gender == 0 else "male"
                        result = np.append(result, f"gender: {gender_text}")

                        age = predict_age(img_infer_pil)[0][0]
                        age_str = "%.2f" % age
                        result = np.append(result, f"age: {age_str}")
                        for i in range(0, len(result)):
                            image = cv2.putText(
                                image,
                                result[i],
                                (x1, y1 + i * 40),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                1,
                                (255, 255, 255),
                                1,
                                cv2.LINE_AA,
                            )

        cv2.imshow("Frame", image)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
cap.release()
cv2.imshow("Frame", image)

1/1 [==============================] - 0s 114ms/step


KeyboardInterrupt: 